In [3]:
import pandas as pd
import requests
import boto3
import os
from time import time

''' 
I want to create two lists. offset will be the index of getting the data, 
while limit will be the index of where to stop getting the data for the csv.
i will start off at 0 for offset. The limit will be 10000. 
For example if a row count has 82,426 each csv file should have 10,000 rows 
leading up to 82,426. The last file should have 2,426.
'''
credentials_path = "/Users/httran/Documents/Programming/Credentials_Repo/"
credentials = pd.read_csv(credentials_path + 'user_credentials.csv')
aws_key = str(credentials['Access key ID'][0])
aws_secret_key = credentials['Secret access key'][0]

s3 = boto3.client('s3', region_name='us-east-1', aws_access_key_id=aws_key, aws_secret_access_key=aws_secret_key)

'''Create the bucket'''
#s3.create_bucket(Bucket='htran-covid-project')

'''Get the count of rows for the dataset. Get the limit and offset parameters. 
Get the limit and offset parameter for each url. Extract that and convert to a csv.
Upload that csv to the s3 bucket. Continue until we reach the end of the list'''

num_rows_url = 'https://data.cdc.gov/resource/9bhg-hcku.json?$select=count(state)'
num_rows_result = requests.get(num_rows_url)
num_rows_data = num_rows_result.json()
num_rows = int(num_rows_data[0]['count_state'])

# I don't like the way this is formatted but we can work on it later
# i = 0
# offset = []
# limit = 10000
# while i < num_rows:
#     offset.append(i)
#     i += 10000
# offset[-1] = num_rows

#example url: https://soda.demo.socrata.com/resource/earthquakes.json?$limit=5&$offset=0$order=data_as_of
# for index_pos in offset:
t0 = time()
url = f'https://data.cdc.gov/resource/9bhg-hcku.json?$limit=50000'
result = requests.get(url)
data = result.json()
df = pd.DataFrame(data)
print(df.columns)
file_name = f'covid19data.csv'
df.to_csv(file_name)
s3.upload_file(file_name, Bucket='htran-covid-project', Key=file_name)
t1 = time()
total_time = round((t1 - t0),2)
print(f'Total time to upload {file_name} was {total_time} seconds.')
# if os.path.exists(file_name):
#     os.remove(file_name)
# else:
#     continue

Index(['data_as_of', 'start_date', 'end_date', 'group', 'state', 'sex',
       'age_group', 'covid_19_deaths', 'total_deaths', 'pneumonia_deaths',
       'pneumonia_and_covid_19_deaths', 'influenza_deaths',
       'pneumonia_influenza_or_covid', 'footnote', 'year', 'month'],
      dtype='object')
Total time to upload covid19data.csv was 4.07 seconds.


In [9]:
df = pd.read_csv('covid19data.csv',index_col = 0)

In [10]:
df.head()

,data_as_of,start_date,end_date,group,state,sex,age_group,covid_19_deaths,total_deaths,pneumonia_deaths,pneumonia_and_covid_19_deaths,influenza_deaths,pneumonia_influenza_or_covid,footnote,year,month
0,2022-03-02T00:00:00.000,2020-01-01T00:00:00.000,2022-02-26T00:00:00.000,By Total,United States,All Sexes,All Ages,944650.0,7297815.0,830505.0,489420.0,10462.0,1294658.0,NaN,NaN,NaN
1,2022-03-02T00:00:00.000,2020-01-01T00:00:00.000,2022-02-26T00:00:00.000,By Total,United States,All Sexes,Under 1 year,224.0,40604.0,476.0,27.0,27.0,699.0,NaN,NaN,NaN
2,2022-03-02T00:00:00.000,2020-01-01T00:00:00.000,2022-02-26T00:00:00.000,By Total,United States,All Sexes,0-17 years,865.0,72039.0,1407.0,241.0,198.0,2228.0,NaN,NaN,NaN
3,2022-03-02T00:00:00.000,2020-01-01T00:00:00.000,2022-02-26T00:00:00.000,By Total,United States,All Sexes,1-4 years,101.0,7562.0,281.0,26.0,67.0,423.0,NaN,NaN,NaN
4,2022-03-02T00:00:00.000,2020-01-01T00:00:00.000,2022-02-26T00:00:00.000,By Total,United States,All Sexes,5-14 years,268.0,12071.0,411.0,93.0,81.0,667.0,NaN,NaN,NaN


In [11]:
pd.Timestamp(df['data_as_of'])

TypeError: Cannot convert input [0        2022-03-02T00:00:00.000
1        2022-03-02T00:00:00.000
2        2022-03-02T00:00:00.000
3        2022-03-02T00:00:00.000
4        2022-03-02T00:00:00.000
                  ...           
49995    2022-03-02T00:00:00.000
49996    2022-03-02T00:00:00.000
49997    2022-03-02T00:00:00.000
49998    2022-03-02T00:00:00.000
49999    2022-03-02T00:00:00.000
Name: data_as_of, Length: 50000, dtype: object] of type <class 'pandas.core.series.Series'> to Timestamp

In [12]:
print(type(df['data_as_of'][0]))

<class 'str'>


In [15]:
print(sorted(df.columns))

['age_group', 'covid_19_deaths', 'data_as_of', 'end_date', 'footnote', 'group', 'influenza_deaths', 'month', 'pneumonia_and_covid_19_deaths', 'pneumonia_deaths', 'pneumonia_influenza_or_covid', 'sex', 'start_date', 'state', 'total_deaths', 'year']
